# CCA Project

### Load Data and Libraries

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline 

import xlrd

from apyori import apriori

In [2]:
data = pd.read_excel('Online Retail.xlsx')

### Explore Data

In [3]:
data.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [6]:
# Number of unqiue invoices
data.InvoiceNo.nunique()

25900

In [7]:
# Number of unique Data Items - slight descrepancy between code and description but not too bad
print(data.StockCode.nunique())
print(data.Description.nunique())

4070
4223


In [8]:
# Top 20 items have support of around 5-10%
data.groupby('Description').InvoiceNo.count().sort_values(ascending = False).head(20)

Description
WHITE HANGING HEART T-LIGHT HOLDER    2369
REGENCY CAKESTAND 3 TIER              2200
JUMBO BAG RED RETROSPOT               2159
PARTY BUNTING                         1727
LUNCH BAG RED RETROSPOT               1638
ASSORTED COLOUR BIRD ORNAMENT         1501
SET OF 3 CAKE TINS PANTRY DESIGN      1473
PACK OF 72 RETROSPOT CAKE CASES       1385
LUNCH BAG  BLACK SKULL.               1350
NATURAL SLATE HEART CHALKBOARD        1280
POSTAGE                               1252
JUMBO BAG PINK POLKADOT               1251
HEART OF WICKER SMALL                 1237
JAM MAKING SET WITH JARS              1229
JUMBO STORAGE BAG SUKI                1214
PAPER CHAIN KIT 50'S CHRISTMAS        1210
JUMBO SHOPPER VINTAGE RED PAISLEY     1202
LUNCH BAG CARS BLUE                   1197
LUNCH BAG SPACEBOY DESIGN             1192
JAM MAKING SET PRINTED                1182
Name: InvoiceNo, dtype: int64

### Quick Data Prep and Market-Basket with Apriori

In [9]:
# Use only 400,000 clean lines
df = data.dropna().iloc[0:400000]

In [10]:
# number of unique invoices and items
print(df.InvoiceNo.nunique())
print(df.Description.nunique())

21823
3894


In [11]:
# Support 5-10%
df.groupby('Description').InvoiceNo.count().sort_values(ascending = False).head(20)

Description
WHITE HANGING HEART T-LIGHT HOLDER    2052
REGENCY CAKESTAND 3 TIER              1885
JUMBO BAG RED RETROSPOT               1641
PARTY BUNTING                         1411
ASSORTED COLOUR BIRD ORNAMENT         1404
LUNCH BAG RED RETROSPOT               1344
SET OF 3 CAKE TINS PANTRY DESIGN      1220
POSTAGE                               1176
LUNCH BAG  BLACK SKULL.               1113
PACK OF 72 RETROSPOT CAKE CASES       1074
SPOTTY BUNTING                        1022
LUNCH BAG SPACEBOY DESIGN             1014
LUNCH BAG CARS BLUE                   1008
PAPER CHAIN KIT 50'S CHRISTMAS         993
NATURAL SLATE HEART CHALKBOARD         989
HEART OF WICKER SMALL                  985
LUNCH BAG PINK POLKADOT                971
JAM MAKING SET WITH JARS               961
REX CASH+CARRY JUMBO SHOPPER           943
LUNCH BAG SUKI DESIGN                  943
Name: InvoiceNo, dtype: int64

In [12]:
df = df[['InvoiceNo', 'Description']]

In [25]:
# Put transaction data into list of list form for use with Apriori algorithm
# SLOW: Should probably pre-allocate the list (for 100,000 items was only 30s)
import time

start = time.time()

invoices = list(df.InvoiceNo.unique())

records = []
for i in range(len(invoices)):
    records.append(list(df.Description[df.InvoiceNo == invoices[i]]))

end = time.time()

print(end-start)

485.76213932037354


In [33]:
# Play with rules and fit algorithm
association_rules = apriori(records, min_support=0.015, min_confidence=0.3, min_lift=3, min_length=2)  
association_results = list(association_rules) 

In [34]:
# How many pairs met the criteria?
print(len(association_results))

49


In [35]:
# Print details for the 49 pairs

for item in association_results[0:49]:
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    print("Support: " + str(item[1]))
    print("Confidence: " + str(item[2][0][2]))
    print("Lift: " + str(item[2][0][3]))
    print("=====================================")

Rule: PACK OF 72 RETROSPOT CAKE CASES -> 60 TEATIME FAIRY CAKE CASES
Support: 0.01516748384731705
Confidence: 0.5038051750380518
Lift: 10.622744284884448
Rule: ALARM CLOCK BAKELIKE PINK -> ALARM CLOCK BAKELIKE GREEN
Support: 0.015579892773679146
Confidence: 0.42875157629255994
Lift: 15.06706223741149
Rule: ALARM CLOCK BAKELIKE GREEN -> ALARM CLOCK BAKELIKE RED 
Support: 0.024011364157081976
Confidence: 0.6607818411097099
Lift: 16.257319186625928
Rule: ALARM CLOCK BAKELIKE PINK -> ALARM CLOCK BAKELIKE RED 
Support: 0.01828346240205288
Confidence: 0.642512077294686
Lift: 15.807825324466664
Rule: DOLLY GIRL LUNCH BOX -> SPACEBOY LUNCH BOX 
Support: 0.019291573110938
Confidence: 0.6790322580645162
Lift: 21.01917867764814
Rule: GARDENERS KNEELING PAD CUP OF TEA  -> GARDENERS KNEELING PAD KEEP CALM 
Support: 0.020803739174265683
Confidence: 0.7275641025641025
Lift: 21.31225692651867
Rule: GREEN REGENCY TEACUP AND SAUCER -> PINK REGENCY TEACUP AND SAUCER
Support: 0.021445264170828943
Confiden